In [1]:
from infinipy.stateblock import StateBlock
import uuid

from infinipy.statement import Statement,CompositeStatement
from infinipy.transformer import Transformer,CompositeTransformer
from infinipy.affordance import Affordance
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Union, Tuple

In [2]:
#we define multiple entities we avoid the spatial component for now
# character 
# door
# key

@dataclass
class Door(StateBlock):
    is_open: bool = False  # Attribute to indicate if the door is open
    is_locked: bool = False  # Attribute to indicate if the door is locked
    def __init__(self, key_id: str,is_open:bool,is_locked:bool, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.key_id = key_id  # Identifier of the key that can open the door
        self.is_open = is_open
        self.is_locked = is_locked

    def __post_init__(self):
        super().__post_init__()  # Call the post-init of StateBlock
        # Additional initialization can be added here if needed
key = StateBlock(
    id=str(uuid.uuid4()),
    owner_id="key_owner",  # Identifier of the key's owner or location
    name="Key",
    blocks_move=False,  # A key doesn't block movement
    blocks_los=False,  # A key doesn't block line of sight
    can_store=True,
    can_be_stored=False,
    can_act=True,
    can_move=False,
    can_be_moved=True,
    position=(0, 0, 0),  # Position of the key in the environment
)

door = Door(
    id=str(uuid.uuid4()),
    owner_id="door_owner",  # Identifier of the door's owner or location
    name="Door",
    blocks_move=True,  # A closed door blocks movement
    blocks_los=False,  # A door doesn't generally block line of sight
    can_store=False,
    can_be_stored=False,
    can_act=False,
    can_move=False,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the door in the environment
    is_open=False,  # Custom attribute indicating if the door is open
    is_locked=True,  # Custom attribute indicating if the door is locked
    key_id=key.id,  # Custom attribute indicating the key that can open the door
)

character = StateBlock(
    id=str(uuid.uuid4()),
    owner_id="character_owner",  # Identifier of the character's owner or location
    name="Character",
    blocks_move=True,  # A character blocks movement
    blocks_los=True,  # A character blocks line of sight
    can_store=True,
    can_be_stored=False,
    can_act=True,
    can_move=True,
    can_be_moved=False,
    position=(0, 0, 0),  # Position of the character in the environment
)



In [3]:
#boolean conditions of the entities 

def is_open_condition(door: StateBlock):
    return door.is_open
def is_locked_condition(door: StateBlock):
    return door.is_locked

def has_key_condition(character: StateBlock):
    key_id = key.id
    for item in character.inventory:
        if item.id == key_id:
            return True
    return False
    

def is_pickable(key: StateBlock):
    return key.can_be_moved

def can_pick(chararacter:StateBlock):
    return chararacter.can_store

def has_inventory_space(character: StateBlock):
    return len(character.inventory) < character.inventory_size


In [4]:
#statements corresponding

statements = {
    "IsOpen": Statement(
    name="IsOpen",
    description="is open",
    callable=is_open_condition,
    usage="target"
    ),
    "IsLocked": Statement(
    name="IsLocked",
    description="is locked",
    callable=is_locked_condition,
    usage="target"),
    "HasKey": Statement(
    name="HasKey",
    description="has key",
    callable=has_key_condition,
    usage="source"),
    "IsPickable": Statement(
    name="IsPickable",
    description="is pickable",
    callable=is_pickable,
    usage="target"),
    "CanPick": Statement(
    name="CanPick",
    description="can pick",
    callable=can_pick,
    usage="source"),
    "HasInventorySpace": Statement(
    name="HasInventorySpace",
    description="has inventory space",
    callable=has_inventory_space,
    usage="source"),
}

In [5]:
from infinipy.actions import Action

In [6]:
closed_locked_door = CompositeStatement([(statements["IsOpen"],False), (statements["IsLocked"],True)])
open_unlocked_door = CompositeStatement([(statements["IsOpen"],True), (statements["IsLocked"],False)])
closed_unlocked_door = CompositeStatement([(statements["IsOpen"],False), (statements["IsLocked"],False)])
open_locked_door = CompositeStatement([(statements["IsOpen"],True), (statements["IsLocked"],True)])

In [7]:
cleaned = open_unlocked_door.remove_intersection(closed_unlocked_door)

In [8]:
for s in closed_unlocked_door.substatements:
    print(s[0].name,s[1])

IsLocked_target False
IsOpen_target False


In [9]:
for s in open_unlocked_door.substatements:
    print(s[0].name,s[1])

IsLocked_target False
IsOpen_target True


In [10]:
for s in cleaned.substatements:
    print(s[0].name,s[1])

IsOpen_target True


In [11]:
CompositeStatement.from_composite_statements([closed_unlocked_door])

In [12]:
open_door_action = Action(name= "character_open_door",
                          prerequisites= [closed_unlocked_door],
                          consequences= [CompositeStatement([(statements["IsOpen"],True)])],
                          source_block= character,
                          target_block= door)

close_door_action = Action(name= "character_close_door",
                            prerequisites= [open_unlocked_door],
                            consequences= [CompositeStatement([(statements["IsOpen"],False)])],
                            source_block= character,
                            target_block= door)

lock_door_action = Action(name= "character_lock_door",
                            prerequisites= [CompositeStatement([(statements["IsLocked"],False)]),
                                            CompositeStatement([(statements["HasKey"],True)])],
                            consequences= [CompositeStatement([(statements["IsLocked"],True)]),],
                            source_block= character,
                            target_block= door)

unlock_door_action = Action(name= "character_unlock_door",
                            prerequisites= [CompositeStatement([(statements["IsLocked"],True)]),
                                            CompositeStatement([(statements["HasKey"],True)])],
                            consequences= [CompositeStatement([(statements["IsLocked"],False)])],
                            source_block= character,
                            target_block= door)
pick_key_action = Action(name= "character_pick_key",
                            prerequisites= [CompositeStatement([(statements["IsPickable"],True)]),
                                            CompositeStatement([(statements["HasInventorySpace"],True)])],
                            consequences= [CompositeStatement([(statements["IsPickable"],False)]),
                                           CompositeStatement([(statements["HasKey"],True)])],
                            source_block= character,
                            target_block= key)

In [26]:
open_door_action.consequences.name

'CompositeStatement()'

In [13]:
from infinipy.options import Option

In [14]:
option = Option()

In [15]:
pick_key_action.pre_dict.keys()

dict_keys([])

In [16]:
option.append(pick_key_action)

key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', None)
key (None, '551407db-4b87-494f-bdab-084088b499db')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', '551407db-4b87-494f-bdab-084088b499db')


In [17]:
option.append(pick_key_action)
option.append(unlock_door_action)
option.append(open_door_action)

key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', None)
key (None, '551407db-4b87-494f-bdab-084088b499db')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', '551407db-4b87-494f-bdab-084088b499db')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', None)
key (None, '3b131201-9af4-4e5c-b90c-afe66f89fcc3')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', '3b131201-9af4-4e5c-b90c-afe66f89fcc3')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', None)
key (None, '3b131201-9af4-4e5c-b90c-afe66f89fcc3')
key ('694fe19a-bc9a-4f2d-a40d-5b389447f0cc', '3b131201-9af4-4e5c-b90c-afe66f89fcc3')


In [18]:
for key,value in option.global_prerequisites.items():
    print(key)
    print(value.name if isinstance(value,CompositeStatement) else value)

In [19]:
for key,value in option.global_consequences.items():
    print(key)
    print(value.name)

In [20]:
option.global_consequences.values()

dict_values([])

In [21]:
door_consequence =option.global_consequences[(None,door.id)]


TypeError: CompositeStatement.__init__() missing 1 required positional argument: 'substatements'

In [ ]:
from infinipy.worldstatement import WorldStatement

In [ ]:
consequence_state_test = WorldStatement.from_dict(option.global_consequences)


In [ ]:
for key,value in consequence_state_test.conditions.items():
    print(key)
    print(value.name)

('76de05d7-5015-4694-9d7a-052d30404041', None)
CompositeStatement(HasInventorySpace_source True, HasKey_source True)
(None, '9c654589-4ec0-4339-90cb-1967247d35c6')
CompositeStatement(IsPickable_target False)
(None, '7a2dce6a-741b-4122-bf29-c914c3de2e6b')
CompositeStatement(IsOpen_target True, IsLocked_target False)


In [ ]:
consequence_world = []
for key,value in option.global_consequences.items():
    consequence_world.append((value,key[0],key[1]))
consequence_state = WorldStatement(consequence_world)

In [ ]:
for key,value in consequence_state.conditions.items():
    print(key)
    print(value.name)

('76de05d7-5015-4694-9d7a-052d30404041', None)
CompositeStatement(HasInventorySpace_source True, HasKey_source True)
(None, '9c654589-4ec0-4339-90cb-1967247d35c6')
CompositeStatement(IsPickable_target False)
(None, '7a2dce6a-741b-4122-bf29-c914c3de2e6b')
CompositeStatement(IsOpen_target True, IsLocked_target False)


In [ ]:
goal = CompositeStatement([(statements["IsOpen"],True)])
goal_state = WorldStatement([(goal,None,door.id)])

In [ ]:
for key,value in goal_state.conditions.items():
    print(key)
    print(value.name)

(None, '7a2dce6a-741b-4122-bf29-c914c3de2e6b')
CompositeStatement(IsOpen_target True)


In [ ]:
consequence_state.validates(goal_state)

True

In [ ]:
goal_state.is_validated_by(consequence_state)

True

In [ ]:
backward_option = Option()

In [ ]:
backward_option.prepend(open_door_action)
backward_option.prepend(unlock_door_action)
backward_option.prepend(pick_key_action)


In [ ]:
for key,value in backward_option.global_prerequisites.items():
    print(key)
    print(value.name if isinstance(value,CompositeStatement) else value)

(None, '7a2dce6a-741b-4122-bf29-c914c3de2e6b')
CompositeStatement(IsLocked_target True, IsOpen_target False)
('76de05d7-5015-4694-9d7a-052d30404041', None)
CompositeStatement(HasInventorySpace_source True)
(None, '9c654589-4ec0-4339-90cb-1967247d35c6')
CompositeStatement(IsPickable_target True)


In [ ]:
for key,value in backward_option.global_consequences.items():
    print(key)
    print(value.name)

(None, '7a2dce6a-741b-4122-bf29-c914c3de2e6b')
CompositeStatement(IsOpen_target True, IsLocked_target False)
('76de05d7-5015-4694-9d7a-052d30404041', None)
CompositeStatement(HasInventorySpace_source True, HasKey_source True)
(None, '9c654589-4ec0-4339-90cb-1967247d35c6')
CompositeStatement(IsPickable_target False)


In [ ]:
pick_key_action.check_prerequisites()

{'result': True,
 'name': 'CompositeStatement(HasInventorySpace_source True, IsPickable_target True)',
 'composite_string': 'The statement HasInventorySpace_source with description has inventory space applied to the source Character is True. AND The statement IsPickable_target with description is pickable applied to the target Key is True.',
 'sub_statements': [{'statement': 'HasInventorySpace_source',
   'result': True,
   'usage': 'source',
   'source': 'Character',
   'target': 'Key',
   'stringout': 'The statement HasInventorySpace_source with description has inventory space applied to the source Character is True.',
   'source_missing': None,
   'target_missing': None},
  {'statement': 'IsPickable_target',
   'result': True,
   'usage': 'target',
   'source': 'Character',
   'target': 'Key',
   'stringout': 'The statement IsPickable_target with description is pickable applied to the target Key is True.',
   'source_missing': None,
   'target_missing': None}],
 'conditions': [True,

In [ ]:

for key, value in pick_key_action.pre_dict.items():
    print(value.name)

CompositeStatement(HasInventorySpace_source True)
CompositeStatement(IsPickable_target True)


In [ ]:
unlock_door_action.check_prerequisites()

AttributeError: 'tuple' object has no attribute 'id'

In [ ]:
lock_door_action.check_consequences()

{'result': False,
 'name': 'CompositeStatement(HasKey_both True, IsLocked_target True)',
 'composite_string': 'The statement HasKey_both with description has key applied to the source Character and the target Door is False. AND The statement IsLocked_target with description is locked applied to the target Door is True.',
 'sub_statements': [{'statement': 'HasKey_both',
   'result': False,
   'usage': 'both',
   'source': 'Character',
   'target': 'Door',
   'stringout': 'The statement HasKey_both with description has key applied to the source Character and the target Door is False.',
   'source_missing': None,
   'target_missing': None},
  {'statement': 'IsLocked_target',
   'result': True,
   'usage': 'target',
   'source': 'Character',
   'target': 'Door',
   'stringout': 'The statement IsLocked_target with description is locked applied to the target Door is True.',
   'source_missing': None,
   'target_missing': None}],
 'conditions': [True, True],
 'sub_results': [False, True]}

In [ ]:
print(door.is_locked,door.is_open)

True False


In [ ]:
test = CompositeStatement([(statements["IsLocked"],True),(statements["IsOpen"],False)])

In [ ]:
test(door)

ValueError: Target block is None but usage is target for statement IsLocked_target

In [ ]:
test = CompositeStatement([(statements["IsOpen"],False),(statements["IsLocked"],True)])

In [ ]:
test2=CompositeStatement([(statements["IsPickable"],True)])

In [ ]:
testjoined = CompositeStatement.from_composite_statements([(test,False),(test2,False)])

In [ ]:
test.substatements

{(<infinipy.statement.Statement at 0x2056f2166e0>, True),
 (<infinipy.statement.Statement at 0x2056f217f70>, False)}

In [ ]:
for statement, condition in testjoined.substatements:
    print(f"{statement.name} {condition}")

IsLocked_source False
IsPickable_source False
IsOpen_source True


In [ ]:
out =test.apply(door)